# Настройка

###  подгрузка библиотек

In [1]:
import psycopg2
import pandas as pd

### данные соединения (удалить после выполнения!!!!)

In [20]:
DBNAME = 
USER = 
PASSWORD = 
HOST = 
PORT = 

### Создаем соединение с заданными параметрами

In [21]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Основная работа

## Предварительный анализ данных

### **Задание 3.1**  


In [35]:
# создаем запрос на вызов всех столбцов и строк из таблицы vacancies схемы public
query_vaccnt = f'''select * /*выбираем все из таблицы vacancies */
            from public.vacancies    /* из схемы public таблицы vacancies */             
        '''

vacancies_df = pd.read_sql_query(query_vaccnt,connection) # создаем дата фрейм из данных запроса
vacancies_df.info() # выводим сведения о дата фреме

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3281498106.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_df = pd.read_sql_query(query_vaccnt,connection) # создаем дата фрейм из данных запроса


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49197 entries, 0 to 49196
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           49197 non-null  int64  
 1   name         49197 non-null  object 
 2   key_skills   40741 non-null  object 
 3   schedule     49197 non-null  object 
 4   experience   49197 non-null  object 
 5   employment   49197 non-null  object 
 6   salary_from  21485 non-null  float64
 7   salary_to    12690 non-null  float64
 8   area_id      49197 non-null  int64  
 9   employer_id  49197 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 3.8+ MB


### **Задание 3.2**  


In [34]:
# создаем запрос из таблицы employers схемы public
query_empcnt = f'''select count(*) employers_count /*подсчитываем количество строк в таблице employers, задаем имя для столбца*/
            from public.employers   /* из схемы public таблицы employers */         
        '''
employers__df = pd.read_sql_query(query_empcnt,connection) # создаем дата фрейм из данных запроса
employers__df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3993865010.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers__df = pd.read_sql_query(query_empcnt,connection) # создаем дата фрейм из данных запроса


,employers_count
0,23501


### **Задание 3.3**  


In [33]:
# создаем запрос из таблицы areas схемы public
query_areacnt = f'''select count(*) regions_count /*подсчитываем количество строк в таблице areas, задаем имя для столбца*/
            from public.areas /* из схемы public таблицы areas */
            '''
areas_df = pd.read_sql_query(query_areacnt,connection) # создаем дата фрейм из данных запроса
areas_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3003094215.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_df = pd.read_sql_query(query_areacnt,connection) # создаем дата фрейм из данных запроса


,regions_count
0,1362


### **Задание 3.4**


In [32]:
# создаем запрос из таблицы industries схемы public
query_indcnt = f'''select count(*) industries_count /*подсчитываем количество строк в таблице industries, задаем имя для столбца*/
            from public.industries /* из схемы public таблицы industries */
            '''

industries_df = pd.read_sql_query(query_indcnt,connection) # создаем дата фрейм из данных запроса
industries_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\2672429431.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industries_df = pd.read_sql_query(query_indcnt,connection) # создаем дата фрейм из данных запроса


,industries_count
0,294


### **Выводы**


В таблице vacancies представленны 49197 вакансий; в столбце salary_from заполнено ~50%, в столбце salary_to заполнено ~ 20%. В таблице employers представлены данные о 23501 работодателей. В таблице areas представлены данные о 1362 регионах. В таблице industries представлены данные о 294 сферах деятельности. Т.о. в схеме public присутвуют данные о 49197 вакансиях от 23501 работодателей по 294 сферам деятельности в 1362 регионах.

## Детальный анализ вакансий

### **Задание 4.1**  

 

In [31]:
# формируем запрос на вывод кол-ва вакансий по регионам, выводим пять регионов с наибольшим кол-вом вакансий отсортированных по убыванию.
query_cnt = f'''select a.name area, count(v.id) cnt /* выводим два столбца: из таблицы area стоблец name; cnt - столбец с кол-вом строк вакансий соответсвующих name */
                from public.areas a /* извлекаем из таблицы areas с присвоением ей алиаса а */
                    join public.vacancies v on a.id = v.area_id /* к таблице areas присоединяем данные из таблицы vacancies, сводим по столбцам с id регионов */
                group by a.name /* группируем по первому столбцу(name) */
                order by 2 desc /* сортируем по второму столбцу(кол-ву вакансий) в порядке убывания */
                limit 5 /* кол-во выводимых значений 5 */
                ''' 

cnt_df = pd.read_sql_query(query_cnt,connection) # создаем дата фрейм из данных запроса
cnt_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\453874910.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_df = pd.read_sql_query(query_cnt,connection) # создаем дата фрейм из данных запроса


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


### **Задание 4.2**  


In [30]:
# формируем запрос для определения у какого кол-ва вакансий из таблицы vacancies заполнено хотя бы один из двух полей с зарплатой
query_vac_count = f'''select count(id) as "count not null" /* выводим сумму строк по столбцу id, присваиваем столбцу алиас */
            from public.vacancies /* указываем схему и таблицу от куда берем данные */
            where salary_from is not null or salary_to is not null /* создаем условие вывода: учитываются только те строки в которых либо salary_from либо salary_to не null */
        '''
        
vac_df = pd.read_sql_query(query_vac_count,connection)# создаем дата фрейм из данных запроса
vac_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\1186021314.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vac_df = pd.read_sql_query(query_vac_count,connection)# создаем дата фрейм из данных запроса


,count not null
0,24073


### **Задание 4.3**  



In [29]:
# создаем запрос для вывода среднего значения нижней и верхней границы зарплатной вилки с округлением до целого числа
query_salary_mean = f'''select round(avg(salary_from)) salary_from, round(avg(salary_to)) salary_to /* берем из таблицы vacancies столбцы salary_from и salary_to, находим среднее по каждому*/
                                                                                                        /* столбцу, округляем значение каждого столбца до целого, даем столбцам алиасы */
            from public.vacancies /* указываем от куда берем данные */
            
        '''
        
salary_mean_df = pd.read_sql_query(query_salary_mean,connection)# создаем дата фрейм из данных запроса
salary_mean_df# выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\2905387058.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_mean_df = pd.read_sql_query(query_salary_mean,connection)# создаем дата фрейм из данных запроса


,salary_from,salary_to
0,71065.0,110537.0


### **Задание 4.4**  



In [28]:
# создаем запрос для вывода кол-ва вакансий для каждого сочетания типа рабочего графика и типа трудоустройства. Выводим пару находящуюся на втором месте по популярности.
query_shedemp = f'''select schedule, employment, count(id) /* выбираем столбцы типа графика(schedule), типа трудоустройства(employment), кол-во строк по id */
            from public.vacancies /* из схемы public и таблицы vacancies */
            group by 1, 2 /* группируем вначале по первому столбцу, затем по второму */
            order by 3 desc /* упорядочиваем по третьему столбцу в порядке убывания */
            offset 1 /* исключаем первую строку из вывода */
            limit 1 /* ограничиваем вывод одной строкой */
            
        '''
shedemp_df = pd.read_sql_query(query_shedemp,connection) # создаем дата фрейм из данных запроса
shedemp_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\1396902071.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  shedemp_df = pd.read_sql_query(query_shedemp,connection) # создаем дата фрейм из данных запроса


,schedule,employment,count
0,Удаленная работа,Полная занятость,7802


### **Задание 4.5**  


In [27]:
# пишем запрос на вывод всех вариантов требуемого опыта в порядке возрастания кол-ва вакансий, в которых указан данный вариант.
query_exp = f'''select experience, count(id) /* выбираем столбец опыт работы(experience), столбец с кол-вом строк по столбцу id */
            from public.vacancies /* из схемы public таблицы vacancies */
            group by 1 /* группируем по первому столбцу (experience) */
            order by 2 asc /* сортируем по второму столбцу в порядке возрастания */
            
            
        '''
exp_df = pd.read_sql_query(query_exp,connection)# создаем дата фрейм из данных запроса
exp_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\2998081855.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_df = pd.read_sql_query(query_exp,connection)# создаем дата фрейм из данных запроса


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


### Выводы

В предоставленных данных наибольшее кол-во вакансий представлено по регионам: Москва 5333, Санкт-Петербург 2851, Минск	2112, Новосибирск 2006, Алматы 1892. На эти пять регионов приходится 1/5 всех вакансий (из 1362 регионов). На Москву ~ 1/12. У 24073 вакансий заполнено хотя бы одно поле зарплатной вилки (верхняя и/или нижняя), что составляет примерно половину всех вакансий. Средее значение нижнего уровня зарплатной вилки 71065.0, среднее значение верхнего уровня зарплатной вилки 110537.0, нужно иметь ввиду, что нижний уровень есть только у половины вакансий, а верхний у около 20% вакансий, следует посмотреть наличие экстремальных значений и их влияние на среднее значение.  Наибольшее кол-во вакансий из многообразия комбинаций типа рабочего графика и типа трудоустройства является удаленная работа при полной занятости - 7802 вакансий. Наименее распространены вакансии с запросом на опыт работы более 6 лет - 1337 (2,7%), наиболее распространенны (более половины всех вакансий) с запросом на опыт работы от 1 года до 3 лет - 26152 (53%). Запрос от 3 до 6 лет занимает вторую позицию и составил 14511(29,5%); без опыта работы - 7197 (14,6%).

## Анализ работодателей

### **Задание 5.1**  


In [26]:
# создаем запрос который выведет работодателей по кол-ву вакансий в порядке убывания.
query_employers = f'''select e.name employer, count(*) count /* выбираем столбец name из таблицы employers присвоив алиас employer, выводим кол-во строк присваиваем алиас count */
                from public.employers e /*  из схемы public и таблицы employers */
                    join public.vacancies v on e.id = v.employer_id /* объединяем с таблицей vacancies по id работодателя */
                group by e.name /* группируем по столбцу name */
                order by 2 desc /* упорядывачаем по второму столбцу в порядке убывания */
                limit 5 /* выводим только пять первый строк */
                '''

employers_df = pd.read_sql_query(query_employers,connection)# создаем дата фрейм из данных запроса
employers_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3038158672.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_df = pd.read_sql_query(query_employers,connection)# создаем дата фрейм из данных запроса


,employer,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


### **Задание 5.2**  


In [25]:

# создаем запрос,который для каждого региона вывеедет кол-во работодателей и вакансий в нем. Выводим регион без вакансий с наибольший кол-вом работодателей.
query_areaempl = f'''select distinct a.name as region, /* выбираем уникальные значения столбеца name таблицы areas */                 
                    count(v.id) as vacation_count, /* выводим столбец с кол-вом вакансий с алиасом vacation_count */
                    count(e.id) as employers_count /* выводим столбец с кол-вом работодателей с алиасом employers_count */
                     
                    from public.areas a /* из схемы public таблицы areas */
                      left join public.vacancies v on v.area_id = a.id /* объединяем слева с таблицей vacancies по id региона */
                      left join public.employers e on e.area = a.id  /* объединяем слева с таблице employers по id региона */
                       
                    group by 1 /* группируем по первому столбцу */
                    having count(v.id) = 0 and count(e.id) !=0 /* выводим только те строкее в которых кол-во вакансий равно 0 */
                    order by 3 desc /* сортируем по столцу с кол-вом работодателей по убыванию */
                                       
                '''
                
areaemp_df = pd.read_sql_query(query_areaempl,connection)# создаем дата фрейм из данных запроса
areaemp_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\1874580880.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areaemp_df = pd.read_sql_query(query_areaempl,connection)# создаем дата фрейм из данных запроса


,region,vacation_count,employers_count
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
181,Шардара,0,1
182,Швеция,0,1
183,Щучье,0,1
184,Эквадор,0,1


### **Задание 5.3**  


In [24]:
# создаем запрос который выведет для каждого работодателя кол-во регионов, в которых он публикует вакансии. Выведем максимальное значение из получившегося списка.
query_areaemployers = f'''select       
                            e.name as employer, /* выбираем столбец name с алиасом employer */
                            count(distinct(v.area_id)) as regions_count /* выведем кол-во уникальных регионов с алиасом regions_count */
                        from
                            public.employers e /* из схемы public таблицы employers */
                            join public.vacancies v  on e.id = v.employer_id  /* объединямем с таблицей vacancies по id работодателя */
        
                        group by 1 /* группируем по первому столбцу */
                        Order by 2 desc   /* сортируем по второму столбцу в порядке убывания */
                    
                '''
                
areaemployers_df = pd.read_sql_query(query_areaemployers,connection)# создаем дата фрейм из данных запроса
areaemployers_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\2696396640.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areaemployers_df = pd.read_sql_query(query_areaemployers,connection)# создаем дата фрейм из данных запроса


,employer,regions_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


### **Задание 5.4**  


In [23]:
# создаем запрос для подсчета кол--ва работодателей, у которых не указана сфера деятельности.
query_empind = f'''select  count(e.id) without_industry /* выбираем столбец с кол-вом id с алиасом without_industry */

                    from public.employers e /* из схемы public таблицы employers */
                        left join public.employers_industries es on es.employer_id = e.id /* объединяем слева с таблицей employers_industries по id работодателя */
                    where es.industry_id is null /* учитываем только те строки гд не указана сфера деятельности */
                    
                '''
                
empid_df = pd.read_sql_query(query_empind,connection)# создаем дата фрейм из данных запроса
empid_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3361100310.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empid_df = pd.read_sql_query(query_empind,connection)# создаем дата фрейм из данных запроса


,without_industry
0,8419


### **Задание 5.5**  


In [22]:
# создаем запрос чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 
query_empindcount = f'''select  e.name employer, /* выбираем столбец name таблицы employers с алиасом employer */
                        count(es.industry_id) industries_count /* выбираем столбец с кол-вом работ сфер деятельности с алиасом industries_count */

                    from public.employers e /* из схемы public таблицы employers */
                        left join public.employers_industries es on es.employer_id = e.id /* объединяем слева с таблицей employers_industries по id работодателя */
                    group by 1 /* группируем по первому столбцу */
                    having count(es.industry_id) = 4 /* выводим только тех работодателей у которых указано 4 сферы деятельности */
                    order by 1 asc /* упорядывачием по первому столбцу в порядке возрастания */
                    offset 2 /* исключаем две первые строке из вывода */
                    limit 1 /* выводим только одну строку */
                '''
                
empindcount_df = pd.read_sql_query(query_empindcount,connection)# создаем дата фрейм из данных запроса
empindcount_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\175167703.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empindcount_df = pd.read_sql_query(query_empindcount,connection)# создаем дата фрейм из данных запроса


,employer,industries_count
0,2ГИС,4


### **Задание 5.6**  



In [21]:
# создаем запрос на вывод кол-ва работодателей у которых в качестве сферы деятельности указана "Разработка программного обеспечения"
query_empinddev = f'''select  count(es.employer_id) employers_count /* выбираем столбец с кол-вом работодателей с алиасом employers_count */

                    from public.employers_industries es /* из схемы public таблицы employers_industries */
                        join public.industries i on es.industry_id = i.id /* объединяем с таблицей industries по id сферы деятельности */
                        
                    where i.name like 'Разработка программного обеспечения' /* учитываем только те строке где указано 'Разработка программного обеспечения' */
                '''
                
empinddev_df = pd.read_sql_query(query_empinddev,connection)# создаем дата фрейм из данных запроса
empinddev_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\2267809062.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empinddev_df = pd.read_sql_query(query_empinddev,connection)# создаем дата фрейм из данных запроса


,employers_count
0,3553


### **Задание 5.7**  


In [20]:
# создаем запрос который выведе кол-во вакансий компании Яндекс в городах-миллиониках по каждому городу и общее кол-во вакансий Яндекса по этим городам
query_yandex = f'''select a.name city, /* выбираем столбец name таблицы areas с алиасом city */
                        count(v.id) vacancies_count /* выбираем столбец с кол-вом вакансий из таблицы vacancies с алиасом vacancies_count */

                    from public.vacancies v /* из схемы public таблицы vacancies */
                        left join public.areas a on a.id = v.area_id /* объединяем слева с таблицей areas по id региона */
                        left join public.employers e on v.employer_id = e.id /* объединяем слева с таблицей employers по id работодателя */

                    where e.name = 'Яндекс' and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 
                                       'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 
                                       'Омск', 'Краснодар', 'Воронеж',  'Пермь', 'Волгоград') /* создаем условие выборки: Яндекс и города-миллионики */
                         
                    group by e.name, a.name /* группируем вначале по столбцу name таблицы employers, затем по столбцу name таблицы areas */
                    
                    union all /* присоединяем результат подзапроса */
                    
                    select 'Total', count(v.id) cnt /* создаем значение Total в первом столбце, выбираем столбец с кол-вом вакансий */
                    
                    from public.vacancies v /* из схемы public таблицы vacancies */
                        left join public.areas a on a.id = v.area_id /* объединяем слева с таблицей areas по id региона */
                        left join public.employers e on v.employer_id = e.id  /* объединяем слева с таблицей employers по id работодателя */

                    where e.name = 'Яндекс' and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 
                                       'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 
                                       'Омск', 'Краснодар', 'Воронеж',  'Пермь', 'Волгоград') /* создаем условие выборки: Яндекс и города-миллионики */
                                       
                    order by vacancies_count /* упорядывачиваем по столбцу vacancies_count в порядке возрастания */
                '''
                
yandex_df = pd.read_sql_query(query_yandex,connection)# создаем дата фрейм из данных запроса
yandex_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\206033401.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yandex_df = pd.read_sql_query(query_yandex,connection)# создаем дата фрейм из данных запроса


,city,vacancies_count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


###  Выводы

ТОП-5 работодалей по кол-ву вакансий: Яндекс 1933, Ростелеком 491, Тинькофф	444, СБЕР 428, Газпром нефть 331. ТОП-5 регионов без вакансий по кол-ву работодателей: Россия 410,Казахстан 207, Московская область 75, Краснодарский край 19, Беларусь 18. В таблице регионов регионы не равновесные, представлены как регионы так и субрегионы.  ТОП-5 работодателей по кол-ву регионов где они публикуются: Яндекс 181, Ростелеком 152, Спецремонт 116, Поляков Денис Иванович 88, ООО ЕФИН 71. Кол-во работодателей, у которых не указана сфера деятельности 8419. У компании 2ГИС указано 4 сферы деятельности. Кол-во работодателей у которых в качестве сферы деятельности указано 'Разработка программного обеспечения' - 3553. Регионы в которых Яндекс выставил вакансии: Омск 21,Челябинск 22, Красноярск 23, Волгоград 24, Пермь 25, Казань 25, Ростов-на-Дону 25, Уфа 26, Самара 26, Краснодар 30, Воронеж 32, Новосибирск 35, Нижний Новгород 36, Екатеринбург 39, Санкт-Петербург 42, Москва 54, Total 485.

## Предметный анализ

### **Задание 6.1**  


In [19]:
# создаем запрос для вывода кол-ва вакансий имеющих отношение к данным
query_data = f'''
                select count(v.id) cnt_data_vac /* выбираем столбец с кол-вом вакансий */
                
                from public.vacancies v /* из схемы public таблицы vacancies */
                
                where v.name ilike '%data%' or v.name ilike '%данн%' /* считаем только строки с указаным условием */

            '''
            
data_df = pd.read_sql_query(query_data,connection) # создаем дата фрейм из данных запроса
data_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\2803488565.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_df = pd.read_sql_query(query_data,connection) # создаем дата фрейм из данных запроса


,cnt_data_vac
0,1771


### **Задание 6.2**  


In [18]:
# создаем запрос возвращает колво подходящих вакансий для начинающегося дата-сайентиста
query_dsvac = f'''
                select count(v.id) junior_vacations_count /* выбираем столбец с кол-вом вакансий с алиасом junior_vacations_count */
                
                from public.vacancies v /* из схемы public таблицы vacancies */
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        and (v.name ilike '%junior%' or  v.experience = 'Нет опыта' or v.employment = 'Стажировка') /* в название специальности должны присутствовать след.условия; в столбце опыт работы - нет опыта или тип занятости - стажировка */                       
                               
                '''
                
dsvac_df = pd.read_sql_query(query_dsvac,connection) # создаем дата фрейм из данных запроса
dsvac_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3513981165.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsvac_df = pd.read_sql_query(query_dsvac,connection) # создаем дата фрейм из данных запроса


,junior_vacations_count
0,51


### **Задание 6.3**  


In [17]:
# создаем запрос возвращающий кол-во вакансий где в качестве ключевого навыка указан SQL или postgres
query_dsvacsql = f'''
                select count(*) ds_vacations_sql /* выбираем столбец с кол-вом вакансий, с алиасом ds_vacations_sql */
                
                from public.vacancies v /* из схемы public таблицы vacancies */
                
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        and (v.key_skills like '%postgres%' or v.key_skills like '%SQL%') /* создаем условие выборки вначале из числа вакансий, затем из числа навыков */
                                             
                '''
                
dsvacsql_df = pd.read_sql_query(query_dsvacsql,connection) # создаем дата фрейм из данных запроса
dsvacsql_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3672138987.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsvacsql_df = pd.read_sql_query(query_dsvacsql,connection) # создаем дата фрейм из данных запроса


,ds_vacations_sql
0,201


### **Задание 6.4**  


In [16]:
# создаем запрос возвращающий кол-во вакансий DS с ключевым навыком Python
query_dsvacpy = f'''
                select count(*) ds_vacations_python /* выбираем столбец с кол-вом вакансий, с алиасом ds_vacations_python */
                
                from public.vacancies v /* из схемы public таблицы vacancies */
                
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        and (v.key_skills ilike '%python%') /* задаем условие выборки: вначале подходящие вакансии, затем наличие в навыках Питона */
                                             
                '''
                
dsvacpy_df = pd.read_sql_query(query_dsvacpy,connection) # создаем дата фрейм из данных запроса
dsvacpy_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3085387992.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsvacpy_df = pd.read_sql_query(query_dsvacpy,connection) # создаем дата фрейм из данных запроса


,ds_vacations_python
0,351


### **Задание 6.5**  


In [15]:
# создаем запрос возвращающий среднее значение ключевых навыков для вакансий DS
query_avg = f'''
            select
                round(avg(length(v.key_skills) - length(replace(v.key_skills, CHR(9), ''))+1), 2) AS avg_skills_count_for_ds /* выбираем столбец в котором из столбца key_skills достается строка с навыками, разделена по табуляции, выведено среднее значение, округляем до второго знака */
                
                from public.vacancies v /* из схемы public таблицы vacancies */
                
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') /* задаем условие выборки по вакансиям подходящим под DS */
                        
            '''
            
avg_df = pd.read_sql_query(query_avg,connection) # создаем дата фрейм из данных запроса
avg_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\856151448.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_df = pd.read_sql_query(query_avg,connection) # создаем дата фрейм из данных запроса


,avg_skills_count_for_ds
0,6.41


### **Задание 6.6**  



In [14]:
# создаем запрос возвращающий среднюю зарплату для типа работы от 3 до 6 лет
query_salary_ds = f'''
            select DISTINCT v.experience, /*выбираем столбец с уникальными типами упыта работы */
                    round(avg(coalesce((v.salary_from+v.salary_to)/2, salary_from, salary_to)), 0) /*выбираем столбец со средним значение зарплаты, округленное */
                
                
                from public.vacancies v /* из схемы public таблицы vacancies */
                
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') /* задаем условие выборки по вакансиям подходящим под DS */
                
                group by 1 /* группируем по первому столбцу */
                having v.experience ='От 3 до 6 лет' /* условие вывода только значения для опыта работы от 3 до 6 лет */
            '''
            
salary_ds_df = pd.read_sql_query(query_salary_ds,connection) # создаем дата фрейм из данных запроса
salary_ds_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_19904\3132538809.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_ds_df = pd.read_sql_query(query_salary_ds,connection) # создаем дата фрейм из данных запроса


,experience,round
0,От 3 до 6 лет,243115.0


### Выводы

Кол-во вакансий имеющих отношение к данным - 1771. Кол-во подходящих вакансий для начинающегося дата-сайентиста 51. Кол-во вакансий где в качестве ключевого навыка указан SQL или postgres - 201. кол-во вакансий DS с ключевым навыком Python - 351. среднее значение ключевых навыков для вакансий DS - 6,41. Средняя зарплата с для типа опыта "От 3 до 6 лет" - 243115.0

## Итоговые выводы по основной работе

В таблице vacancies представленны 49197 вакансий; в столбце salary_from заполнено ~50%, в столбце salary_to заполнено ~ 20%. В таблице employers представлены данные о 23501 работодателей. В таблице areas представлены данные о 1362 регионах. В таблице industries представлены данные о 294 сферах деятельности. Т.о. в схеме public присутвуют данные о 49197 вакансиях от 23501 работодателей по 294 сферам деятельности в 1362 регионах.  

В предоставленных данных наибольшее кол-во вакансий представлено по регионам: Москва 5333, Санкт-Петербург 2851, Минск	2112, Новосибирск 2006, Алматы 1892. На эти пять регионов приходится 1/5 всех вакансий (из 1362 регионов). На Москву ~ 1/12. У 24073 вакансий заполнено хотя бы одно поле зарплатной вилки (верхняя и/или нижняя), что составляет примерно половину всех вакансий. Средее значение нижнего уровня зарплатной вилки 71065.0, среднее значение верхнего уровня зарплатной вилки 110537.0, нужно иметь ввиду, что нижний уровень есть только у половины вакансий, а верхний у около 20% вакансий, следует посмотреть наличие экстремальных значений и их влияние на среднее значение.  Наибольшее кол-во вакансий из многообразия комбинаций типа рабочего графика и типа трудоустройства является удаленная работа при полной занятости - 7802 вакансий. Наименее распространены вакансии с запросом на опыт работы более 6 лет - 1337 (2,7%), наиболее распространенны (более половины всех вакансий) с запросом на опыт работы от 1 года до 3 лет - 26152 (53%). Запрос от 3 до 6 лет занимает вторую позицию и составил 14511(29,5%); без опыта работы - 7197 (14,6%).  

ТОП-5 работодалей по кол-ву вакансий: Яндекс 1933, Ростелеком 491, Тинькофф	444, СБЕР 428, Газпром нефть 331. ТОП-5 регионов без вакансий по кол-ву работодателей: Россия 410,Казахстан 207, Московская область 75, Краснодарский край 19, Беларусь 18. В таблице регионов регионы не равновесные, представлены как регионы так и их субрегионы, что делает их не репрезентативными.  ТОП-5 работодателей по кол-ву регионов где они публикуются: Яндекс 181, Ростелеком 152, Спецремонт 116, Поляков Денис Иванович 88, ООО ЕФИН 71. Кол-во работодателей, у которых не указана сфера деятельности 8419. У компании 2ГИС указано 4 сферы деятельности. Кол-во работодателей у которых в качестве сферы деятельности указано 'Разработка программного обеспечения' - 3553. Регионы в которых Яндекс выставил вакансии: Омск 21,Челябинск 22, Красноярск 23, Волгоград 24, Пермь 25, Казань 25, Ростов-на-Дону 25, Уфа 26, Самара 26, Краснодар 30, Воронеж 32, Новосибирск 35, Нижний Новгород 36, Екатеринбург 39, Санкт-Петербург 42, Москва 54, Total 485.  

Кол-во вакансий имеющих отношение к данным - 1771. Кол-во подходящих вакансий для начинающегося дата-сайентиста 51. Кол-во вакансий где в качестве ключевого навыка указан SQL или postgres - 201. кол-во вакансий DS с ключевым навыком Python - 351. среднее значение ключевых навыков для вакансий DS - 6,41. Средняя зарплата с для типа опыта "От 3 до 6 лет" - 243115.0



# Дополнительные исследования


### Анализ зарплатной вилки

In [78]:
# количество не нулевых значений, статистика в верхней границы зарплатной вилки
query_zp = f'''
                select count(salary_to) as "not_null_cont", --выводим все значения не null
                max(salary_to), -- выводим максимальное значение
                min(salary_to),  -- выводим минимальное значение
                mode() WITHIN GROUP (ORDER BY salary_to), --выводим моду
                percentile_cont(0.5) WITHIN GROUP (ORDER BY salary_to), --выводим медиану
                ROUND(AVG(salary_to),2) avg -- выводим среднее значение
                
                from public.vacancies --из схемы public таблицы vacancies
                
                where salary_to is not null --условие вывода не нулевых значений
                

'''
zp_up_df = pd.read_sql(query_zp, connection)# создаем дата фрейм
zp_up_df#  возвращаем дата фрейм

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\3292870893.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zp_up_df = pd.read_sql(query_zp, connection)


,not_null_cont,max,min,mode,percentile_cont,avg
0,12690,1000000,60,50000,80000.0,110536.74


In [63]:
# смотрим количественное выражение различных значений
query_zp = f'''
                select 'salary_to=1m' title, count(salary_to) --выводим кол-во максимальных значений
                from public.vacancies --из схемы public таблицы vacancies
                where salary_to = 1000000 --условие вывода только максимальных значений
                
                union all -- присоединяем 
                
                select 'salary_to<1000', count(salary_to) --выводим крайне малые значения
                from public.vacancies --из схемы public таблицы vacancies
                where salary_to < 1000 --условие вывода крайне малые значения
                
                union all -- присоединяем 
                
                select '50000>salary_to<110000', count(salary_to) --выводим кол-во значений между модой и средней
                from public.vacancies --из схемы public таблицы vacancies
                where (salary_to < 110000) and (salary_to > 50000) -- условие вывода
                
                order by 2 asc -- сортируем по кол-ву в порядке возрастания
                
                

'''
zp_up_df = pd.read_sql(query_zp, connection)# создаем дата фрейм
zp_up_df#  возвращаем дата фрейм

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\725735332.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zp_up_df = pd.read_sql(query_zp, connection)


,title,count
0,salary_to=1m,3
1,salary_to<1000,4
2,50000>salary_to<110000,4204


In [76]:
# смотрим строки с экстремальными значениями
query_zp = f'''
                select * --выбрать все строки
                from public.vacancies --из схемы public таблицы vacancies
                where (salary_to = 1000000) or  (salary_to < 1000)    -- диапазон вывода         
                
                order by salary_to desc --упорядывачиваем
                

'''
zp_up_df = pd.read_sql(query_zp, connection)# создаем дата фрейм
zp_up_df#  возвращаем дата фрейм

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\322071925.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zp_up_df = pd.read_sql(query_zp, connection)


,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,53692636,Брокер по продаже апартаментов,Ведение переговоров\tТелефонные переговоры\tРа...,Полный день,От 1 года до 3 лет,Полная занятость,500000.0,1000000,237,1625845
1,50779568,СТО/Директор по разработке,Laravel\tУправление проектами\tPHP\tVue.js\tPo...,Полный день,От 3 до 6 лет,Полная занятость,500000.0,1000000,1,988480
2,66097724,QA Automation Engineer,Тестирование\tHTML\tJavaScript\tGit\tТестирова...,Полный день,От 1 года до 3 лет,Полная занятость,400000.0,1000000,160,703174
3,55502745,"Консультант 1С - бухгалтерия, зарплата",1С: Предприятие 8\t1С программирование\t1С: За...,Полный день,Более 6 лет,Частичная занятость,750.0,850,102,3074036
4,55518035,Computer Vision Engineer,Tensorflow\tOpenCV\tMachine Learning\tPython\t...,Полный день,От 3 до 6 лет,Полная занятость,250.0,350,1,4484124
5,55385660,Программист,Sass\tjQuery\tAjax\tJavaScript\tHTML,Полный день,От 1 года до 3 лет,Полная занятость,NaN,75,23,1729613
6,55592532,Database Administrator (Администратор баз данных),Работа с базами данных\tPostgreSQL\tУдаленная ...,Удаленная работа,От 1 года до 3 лет,Полная занятость,60.0,60,83,1654478


In [66]:
# количество не нулевых значений, статистика в нижней границы зарплатной вилки
query_zp = f'''
                select count(salary_from) "not_null_cont", --выводим все значения не null
                max(salary_from), -- выводим максимальное значение
                min(salary_from), -- выводим минимальное значение
                mode() WITHIN GROUP (ORDER BY salary_from), --выводим моду
                percentile_cont(0.5) WITHIN GROUP (ORDER BY salary_from), --выводим медиану
                ROUND(AVG(salary_from),2) avg -- выводим среднее значение
                
                from public.vacancies --из схемы public таблицы vacancies
                
                where salary_to is not null --условие вывода не нулевых значений
                

'''
zp_up_df = pd.read_sql(query_zp, connection)# создаем дата фрейм
zp_up_df#  возвращаем дата фрейм

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\2999560027.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zp_up_df = pd.read_sql(query_zp, connection)


,not_null_cont,max,min,mode,percentile_cont,avg
0,10102,574000,30,40000,50000.0,70483.58


In [70]:
# количество не нулевых значений, статистика в нижней границы зарплатной вилки
query_zp = f'''
                select 'salary_from=574k' title, count(salary_from)
                from public.vacancies --из схемы public таблицы vacancies
                where salary_from=574000--условие вывода только максимальных значений
                
                union all --присоединяем
                
                select 'salary_from<1000', count(salary_from) --выводим крайне малые значения
                from public.vacancies --из схемы public таблицы vacancies
                where salary_from=1000 -- условие вывода крайне малых значений
                
                union all --присоединяем
                
                select '40000>salary_from<70000', count(salary_from) --выводим кол-во значений между модой и средней
                from public.vacancies --из схемы public таблицы vacancies
                where (salary_from < 70000) and (salary_from > 40000)  -- условие вывода
                
                order by 2 asc -- сортируем по кол-ву в порядке возрастания

'''
zp_up_df = pd.read_sql(query_zp, connection)# создаем дата фрейм
zp_up_df#  возвращаем дата фрейм

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\2405318503.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zp_up_df = pd.read_sql(query_zp, connection)


,title,count
0,salary_from=574k,2
1,salary_from<1000,5
2,40000>salary_from<70000,5267


In [77]:
# смотрим строки с экстремальными значениями
query_zp = f'''
                select * --выбрать все строки
                from public.vacancies --из схемы public таблицы vacancies
                where (salary_from=574000) or (salary_from < 1000)  -- диапазон вывода
                
                order by salary_from desc --упорядывачиваем                       

'''
zp_up_df = pd.read_sql(query_zp, connection)# создаем дата фрейм
zp_up_df#  возвращаем дата фрейм

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\4133429647.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zp_up_df = pd.read_sql(query_zp, connection)


,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55487195,FullStack Developer in Unicorn Company,Java\tReact\tKotlin\tSpring Boot\tAWS\tGCP\tMo...,Полный день,От 3 до 6 лет,Полная занятость,574000,860000.0,160,5973965
1,55487189,Frontend Developer in Unicorn Company,Node.js\tJavaScript\tMongoDB\tAngular\tSpring ...,Полный день,От 3 до 6 лет,Полная занятость,574000,860000.0,160,5973965
2,55230629,PHP-программист (WordPress),JavaScript\tHTML5\tPHP\tCMS Wordpress\tjQuery,Полный день,От 1 года до 3 лет,Полная занятость,900,NaN,2758,6138766
3,55069564,Контент-менеджер,Adobe Photoshop\tНаполнение контентом\tГрамотн...,Удаленная работа,От 1 года до 3 лет,Полная занятость,900,NaN,1007,2848813
4,55502745,"Консультант 1С - бухгалтерия, зарплата",1С: Предприятие 8\t1С программирование\t1С: За...,Полный день,Более 6 лет,Частичная занятость,750,850.0,102,3074036
5,55518035,Computer Vision Engineer,Tensorflow\tOpenCV\tMachine Learning\tPython\t...,Полный день,От 3 до 6 лет,Полная занятость,250,350.0,1,4484124
6,55851928,Программист Python,None,Полный день,От 1 года до 3 лет,Полная занятость,105,215000.0,4,67611
7,51504693,Системный администратор,TCP/IP\tАдминистрирование сетевого оборудовани...,Полный день,Нет опыта,Полная занятость,80,150000.0,2077,44936
8,53203579,Аналитик 1С,None,Полный день,От 1 года до 3 лет,Полная занятость,80,120000.0,99,360319
9,51951335,Системный администратор,None,Полный день,От 3 до 6 лет,Полная занятость,70,NaN,78,43906


#### Выводы

Сведения о зарабатной плате заполнены частично, так же присутствуют экстремальные значения. Экстремальные значения по salary_from и salary_to несвязаны между собой (не относятся к одним строкам). Экстремально малые значения могут быть как следствение частичной конвертации валют, так и намереного внесения таких данных. Данных о зарплатной вилке мало для использования в дальнейшей работе(нерепрезентативны). 

### Выяснить в какую отрасль, регион набирают ДС, долю вакансий джунов

In [87]:
# Работодатели выставляющие вакансии ДС
query_dsregion = f'''
                select e.name, count(v.id) vacations_count 
                
                from public.vacancies v 
                        join public.employers e on e.id = v.employer_id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        
                group by e.name
                        
                union all
                
                select 'total', count(v.id)
                
                 from public.vacancies v 
                        join public.employers e on e.id = v.employer_id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        
                
                order by 2 desc
                limit 10
                '''
                
dsregion_df = pd.read_sql_query(query_dsregion,connection) # создаем дата фрейм из данных запроса
dsregion_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\2034004164.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsregion_df = pd.read_sql_query(query_dsregion,connection) # создаем дата фрейм из данных запроса


,name,vacations_count
0,total,480
1,СБЕР,37
2,Bell Integrator,25
3,Банк ВТБ (ПАО),18
4,VK,15
5,Positive Technologies,11
6,Яндекс,9
7,EvenBet Gaming,9
8,МегаФон,8
9,Andersen,7


In [89]:
# Работодатели выставляющие вакансии ДС для джунов
query_dsemop = f'''
                select e.name, count(v.id) junior_vacations_count 
                
                from public.vacancies v 
                        join public.employers e on e.id = v.employer_id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        and (v.name ilike '%junior%' or  v.experience = 'Нет опыта' or v.employment = 'Стажировка') 
                        
                group by e.name
                
                union all
                
                select 'total', count(v.id)
                
                from public.vacancies v 
                        join public.employers e on e.id = v.employer_id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        and (v.name ilike '%junior%' or  v.experience = 'Нет опыта' or v.employment = 'Стажировка')
                
                order by 2 desc
                limit 10
                '''
                
dsdsemop_df = pd.read_sql_query(query_dsemop,connection) # создаем дата фрейм из данных запроса
dsdsemop_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\4293344931.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsdsemop_df = pd.read_sql_query(query_dsemop,connection) # создаем дата фрейм из данных запроса


,name,junior_vacations_count
0,total,51
1,Банк ВТБ (ПАО),10
2,Comexp,3
3,The Skolkovo Institute of Science and Technology,3
4,SteadyControl,2
5,"СтарЛайн, Научно-производственное объединение",2
6,Диплей,2
7,Seldon,1
8,DNS Головной офис,1
9,VK,1


In [90]:
# По каким регионам происходит распредееление вакансий ДС
query_dsregion = f'''
                select a.name, count(v.id) vacations_count 
                
                from public.vacancies v 
                        join public.areas a on v.area_id = a.id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                         
                        
                group by a.name
                
                union all
                
                select 'total', count(v.id)
                
                  from public.vacancies v 
                        join public.areas a on v.area_id = a.id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                
                order by 2 desc
                limit 10
                '''
                
dsregion_df = pd.read_sql_query(query_dsregion,connection) # создаем дата фрейм из данных запроса
dsregion_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\2209132322.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsregion_df = pd.read_sql_query(query_dsregion,connection) # создаем дата фрейм из данных запроса


,name,vacations_count
0,total,480
1,Москва,217
2,Санкт-Петербург,64
3,Новосибирск,23
4,Нижний Новгород,20
5,Казань,16
6,Алматы,16
7,Минск,9
8,Томск,8
9,Екатеринбург,7


In [91]:
# По каким регионам происходит распредееление вакансий ДС для джунов
query_dsregion = f'''
                select a.name, count(v.id) junior_vacations_count 
                
                from public.vacancies v 
                        join public.areas a on v.area_id = a.id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        and (v.name ilike '%junior%' or  v.experience = 'Нет опыта' or v.employment = 'Стажировка') 
                        
                group by a.name
                
                 union all
                
                select 'total', count(v.id)
                
                  from public.vacancies v 
                        join public.areas a on v.area_id = a.id
                        
                where  (v.name ilike '%data scientist%' 
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%' 
                        or (v.name like '%ML%' and v.name not like '%HTML%') 
                        or v.name ilike '%machine learning%' 
                        or v.name ilike '%машинн%обучен%') 
                        and (v.name ilike '%junior%' or  v.experience = 'Нет опыта' or v.employment = 'Стажировка') 
                
                order by 2 desc
                '''
                
dsregion_df = pd.read_sql_query(query_dsregion,connection) # создаем дата фрейм из данных запроса
dsregion_df # выводим дата фрем

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_8692\3464552671.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dsregion_df = pd.read_sql_query(query_dsregion,connection) # создаем дата фрейм из данных запроса


,name,junior_vacations_count
0,total,51
1,Москва,19
2,Санкт-Петербург,9
3,Новосибирск,5
4,Минск,2
5,Воронеж,2
6,Екатеринбург,2
7,Ростов-на-Дону,2
8,Пермь,2
9,Томск,2


#### Выводы

В рассматриваемый период ДС набирали приимущественно в банковской сфере в Москве. Доля вакансий на позицию джуна около 10%. 

### Сколько вакансий связано со сферой IT

In [22]:
# смотрим в каком кол-ве вакансий какие сферы деятельности указаны
query_vacidcount = f'''
                    select distinct i.name industry_name, count(v.id) count_of_vacancions /*выбираем столбцы с названием сферы деятельности и кол-вом вакансий */
                    
                    from public.vacancies v /*из схемы public таблицы vacancies */
                    join public.employers_industries e on e.employer_id = v.employer_id /*объединяем с таблицей employers_industries */
                    join public.industries i on i.id = e.industry_id /* объединяем с таблицей industries *
                    
                    group by 1
                    having count(v.id) > 1000
                    order by 2 desc
                    
                    '''

vacidcount_df = pd.read_sql_query(query_vacidcount,connection)
vacidcount_df

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_13240\2760453091.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacidcount_df = pd.read_sql_query(query_vacidcount,connection)


,industry_name,count_of_vacancions
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
5,Кадровые агентства,1143
6,Консалтинговые услуги,1137
7,Интернет-провайдер,1095
8,Розничная сеть (продуктовая),1034
9,Фиксированная связь,1028


Более 60% вакансий связаны со сферой IT. 

# Выводы

* В схеме public присутвуют данные о 49197 вакансиях от 23501 работодателей по 294 сферам деятельности в 1362 регионах.  

* В предоставленных данных наибольшее кол-во вакансий: Москва 5333, Санкт-Петербург 2851, Минск	2112, Новосибирск 2006, Алматы 1892. 
    * На эти пять регионов приходится 1/5 всех вакансий (из 1362 регионов). 
    * На Москву ~ 1/12.  

* У 24073 вакансий заполнено хотя бы одно поле зарплатной вилки (верхняя и/или нижняя), что составляет примерно половину всех вакансий. 
    * Присутствуют экстремальные значения. 
    * Экстремальные значения по salary_from и salary_to несвязаны между собой (не относятся к одним строкам). 
    * Экстремально малые значения могут быть как следствение частичной конвертации валют, так и намереного внесения таких данных.  
    
    Данных о зарплатной вилке мало для использования в дальнейшей работе(нерепрезентативны).  

* Наибольшее кол-во вакансий из многообразия комбинаций типа рабочего графика и типа трудоустройства является удаленная работа при полной занятости - 7802 вакансий.

* Более половины всех вакансий с запросом на опыт работы от 1 года до 3 лет - 26152 (53%).

* Наибольшее кол-во вакансий приходится на коммуникационную и банковскую сферу. Компании из этих отраслей выставляют наибольшее кол-во вакансий на одну компанию. 

* Яндекс и Ростелеком размещают вакансии в широком географическом поле.

* В таблице регионов регионы не равновесные, представлены как регионы так и их субрегионы, что делает их не репрезентативными (сложно применимыми в дальнейшем исследовании, нужна группировка).

* Более 60% вакансий связаны со сферой IT. 

* Вакансии имеющие отношение к данным составляют 3,5 % от общего числа вакансий.

* В рассматриваемый период ДС набирали приимущественно в банковской сфере в Москве. Доля вакансий на позицию джуна около 10%.

# Закрытие соединения

In [23]:
connection.close()